In [22]:
import pandas as pd
import numpy as np
df1=pd.read_csv("movies.csv")
df1.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [23]:
df1.shape


(9742, 3)

In [24]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [25]:
df2=pd.read_csv("ratings.csv")
df2.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [26]:
df2=df2.drop('timestamp',axis=1)
df2.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [27]:
df3=df2.groupby('movieId')[['userId']].count()
df3.head()

,userId
movieId,
1,215
2,110
3,52
4,7
5,49


In [28]:
df2=df2.groupby('movieId')[['rating']].mean()
df2.head()

,rating
movieId,
1,3.920930
2,3.431818
3,3.259615
4,2.357143
5,3.071429


In [29]:
df1=df1.merge(df2,on='movieId')
df1.head()

,movieId,title,genres,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.920930
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.431818
2,3,Grumpier Old Men (1995),Comedy|Romance,3.259615
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.357143
4,5,Father of the Bride Part II (1995),Comedy,3.071429


In [30]:
df1=df1.rename(columns={'rating':'avg_rating'})
df1.head()

,movieId,title,genres,avg_rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.920930
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.431818
2,3,Grumpier Old Men (1995),Comedy|Romance,3.259615
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.357143
4,5,Father of the Bride Part II (1995),Comedy,3.071429


In [31]:
df1=df1.merge(df3,on='movieId')
df1.head()

,movieId,title,genres,avg_rating,userId
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.920930,215
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.431818,110
2,3,Grumpier Old Men (1995),Comedy|Romance,3.259615,52
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.357143,7
4,5,Father of the Bride Part II (1995),Comedy,3.071429,49


In [32]:
df1=df1.rename(columns={'userId':'vote_count'})
df1.head()

,movieId,title,genres,avg_rating,vote_count
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.920930,215
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.431818,110
2,3,Grumpier Old Men (1995),Comedy|Romance,3.259615,52
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.357143,7
4,5,Father of the Bride Part II (1995),Comedy,3.071429,49


In [33]:
df1=df1.drop('genres',axis=1)
df1.head()

,movieId,title,avg_rating,vote_count
0,1,Toy Story (1995),3.920930,215
1,2,Jumanji (1995),3.431818,110
2,3,Grumpier Old Men (1995),3.259615,52
3,4,Waiting to Exhale (1995),2.357143,7
4,5,Father of the Bride Part II (1995),3.071429,49


Demographic filtering Recommender system


In [34]:
C=df1['avg_rating'].mean()
C

3.2624482748109656

In [35]:
m= df1['vote_count'].quantile(0.9)
m

27.0

In [36]:
q_movies = df1.copy().loc[df1['vote_count'] >= m]
q_movies.shape

(976, 4)

In [37]:
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['avg_rating']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [38]:
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

In [39]:
q_movies = q_movies.sort_values('score', ascending=False)
q_movies.head(10)

,movieId,title,avg_rating,vote_count,score
277,318,"Shawshank Redemption, The (1994)",4.429022,317,4.337460
659,858,"Godfather, The (1972)",4.289062,192,4.162494
2224,2959,Fight Club (1999),4.272936,218,4.161576
224,260,Star Wars: Episode IV - A New Hope (1977),4.231076,251,4.137000
46,50,"Usual Suspects, The (1995)",4.237745,204,4.123749
257,296,Pulp Fiction (1994),4.197068,307,4.121515
461,527,Schindler's List (1993),4.225000,220,4.119782
1938,2571,"Matrix, The (1999)",4.192446,278,4.110118
897,1196,Star Wars: Episode V - The Empire Strikes Back...,4.215640,211,4.107505
314,356,Forrest Gump (1994),4.164134,329,4.095747


Collobarative filtering

In [40]:
df5=pd.read_csv('ratings.csv')
df5.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [41]:
df5=df5.drop('timestamp',axis=1)
df5.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [43]:
useritem = np.zeros([10000, 10000])
for userid in range(len(df5['userId'])):
    try:
        useritem[df5['userId'][userid]-1][df5['movieId'][userid]-1]=df5['rating'][userid]
    except:
        pass

In [45]:
moviecolumns=[]
for movieval in range(len(useritem)):
    moviecolumns.append("movie "+str(movieval))

In [46]:
mdf=pd.DataFrame(useritem,columns=moviecolumns)
mdf.head()

,movie 0,movie 1,movie 2,movie 3,movie 4,movie 5,movie 6,movie 7,movie 8,movie 9,movie 10,movie 11,movie 12,movie 13,movie 14,movie 15,movie 16,movie 17,movie 18,movie 19,movie 20,movie 21,movie 22,movie 23,movie 24,movie 25,movie 26,movie 27,movie 28,movie 29,movie 30,movie 31,movie 32,movie 33,movie 34,movie 35,movie 36,movie 37,movie 38,movie 39,...,movie 9960,movie 9961,movie 9962,movie 9963,movie 9964,movie 9965,movie 9966,movie 9967,movie 9968,movie 9969,movie 9970,movie 9971,movie 9972,movie 9973,movie 9974,movie 9975,movie 9976,movie 9977,movie 9978,movie 9979,movie 9980,movie 9981,movie 9982,movie 9983,movie 9984,movie 9985,movie 9986,movie 9987,movie 9988,movie 9989,movie 9990,movie 9991,movie 9992,movie 9993,movie 9994,movie 9995,movie 9996,movie 9997,movie 9998,movie 9999
0,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
mdf['movie 0'][0]

4.0

In [49]:
def standerdize(row):
    new_row=(row-row.mean())/(row.max()-row.min())
    return new_row

In [50]:
smdf=mdf.apply(standerdize)
smdf=smdf.fillna(0)
smdf.head()

,movie 0,movie 1,movie 2,movie 3,movie 4,movie 5,movie 6,movie 7,movie 8,movie 9,movie 10,movie 11,movie 12,movie 13,movie 14,movie 15,movie 16,movie 17,movie 18,movie 19,movie 20,movie 21,movie 22,movie 23,movie 24,movie 25,movie 26,movie 27,movie 28,movie 29,movie 30,movie 31,movie 32,movie 33,movie 34,movie 35,movie 36,movie 37,movie 38,movie 39,...,movie 9960,movie 9961,movie 9962,movie 9963,movie 9964,movie 9965,movie 9966,movie 9967,movie 9968,movie 9969,movie 9970,movie 9971,movie 9972,movie 9973,movie 9974,movie 9975,movie 9976,movie 9977,movie 9978,movie 9979,movie 9980,movie 9981,movie 9982,movie 9983,movie 9984,movie 9985,movie 9986,movie 9987,movie 9988,movie 9989,movie 9990,movie 9991,movie 9992,movie 9993,movie 9994,movie 9995,movie 9996,movie 9997,movie 9998,movie 9999
0,0.78314,-0.00755,0.79661,-0.00055,-0.00301,0.79195,-0.00344,-0.00046,-0.001,-0.00923,-0.00514,-0.00092,-0.000625,-0.00138,-0.00078,-0.00644,-0.00506,-0.00148,-0.0048,-0.000937,-0.00622,-0.00232,-0.001,-0.00175,-0.00551,-0.001138,-0.0006,-0.00093,-0.00305,-0.000225,-0.00242,-0.0141,0.0,-0.00935,0.0,-0.00514,0.0,-0.000283,-0.00685,-0.00017,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.01686,-0.00755,-0.00339,-0.00055,-0.00301,-0.00805,-0.00344,-0.00046,-0.001,-0.00923,-0.00514,-0.00092,-0.000625,-0.00138,-0.00078,-0.00644,-0.00506,-0.00148,-0.0048,-0.000937,-0.00622,-0.00232,-0.001,-0.00175,-0.00551,-0.001138,-0.0006,-0.00093,-0.00305,-0.000225,-0.00242,-0.0141,0.0,-0.00935,0.0,-0.00514,0.0,-0.000283,-0.00685,-0.00017,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.01686,-0.00755,-0.00339,-0.00055,-0.00301,-0.00805,-0.00344,-0.00046,-0.001,-0.00923,-0.00514,-0.00092,-0.000625,-0.00138,-0.00078,-0.00644,-0.00506,-0.00148,-0.0048,-0.000937,-0.00622,-0.00232,-0.001,-0.00175,-0.00551,-0.001138,-0.0006,-0.00093,-0.00305,-0.000225,0.09758,-0.0141,0.0,-0.00935,0.0,-0.00514,0.0,-0.000283,-0.00685,-0.00017,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.01686,-0.00755,-0.00339,-0.00055,-0.00301,-0.00805,-0.00344,-0.00046,-0.001,-0.00923,-0.00514,-0.00092,-0.000625,-0.00138,-0.00078,-0.00644,-0.00506,-0.00148,-0.0048,-0.000937,0.59378,-0.00232,-0.001,-0.00175,-0.00551,-0.001138,-0.0006,-0.00093,-0.00305,-0.000225,-0.00242,0.3859,0.0,-0.00935,0.0,-0.00514,0.0,-0.000283,-0.00685,-0.00017,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.78314,-0.00755,-0.00339,-0.00055,-0.00301,-0.00805,-0.00344,-0.00046,-0.001,-0.00923,-0.00514,-0.00092,-0.000625,-0.00138,-0.00078,-0.00644,-0.00506,-0.00148,-0.0048,-0.000937,0.79378,-0.00232,-0.001,-0.00175,-0.00551,-0.001138,-0.0006,-0.00093,-0.00305,-0.000225,-0.00242,-0.0141,0.0,0.79065,0.0,0.79486,0.0,-0.000283,0.59315,-0.00017,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
from sklearn.metrics.pairwise import cosine_similarity
item_similarity=cosine_similarity(smdf.T)

In [52]:
its=pd.DataFrame(item_similarity,columns=moviecolumns)

In [53]:
its.head()

,movie 0,movie 1,movie 2,movie 3,movie 4,movie 5,movie 6,movie 7,movie 8,movie 9,movie 10,movie 11,movie 12,movie 13,movie 14,movie 15,movie 16,movie 17,movie 18,movie 19,movie 20,movie 21,movie 22,movie 23,movie 24,movie 25,movie 26,movie 27,movie 28,movie 29,movie 30,movie 31,movie 32,movie 33,movie 34,movie 35,movie 36,movie 37,movie 38,movie 39,...,movie 9960,movie 9961,movie 9962,movie 9963,movie 9964,movie 9965,movie 9966,movie 9967,movie 9968,movie 9969,movie 9970,movie 9971,movie 9972,movie 9973,movie 9974,movie 9975,movie 9976,movie 9977,movie 9978,movie 9979,movie 9980,movie 9981,movie 9982,movie 9983,movie 9984,movie 9985,movie 9986,movie 9987,movie 9988,movie 9989,movie 9990,movie 9991,movie 9992,movie 9993,movie 9994,movie 9995,movie 9996,movie 9997,movie 9998,movie 9999
0,1.000000,0.402207,0.290753,0.032318,0.302935,0.367715,0.270871,0.129201,0.229635,0.385944,0.316627,0.216495,0.148848,0.174105,0.100803,0.283508,0.327840,0.163600,0.376416,0.143421,0.326941,0.220880,0.160983,0.210378,0.316478,0.167254,0.103787,0.121200,0.183801,0.045618,0.172003,0.499262,0.0,0.497658,0.0,0.281538,0.0,0.133866,0.342266,0.040238,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.402207,1.000000,0.277549,0.104439,0.283069,0.289873,0.223136,0.170743,0.041187,0.411035,0.316674,0.165667,0.192647,0.095708,0.227077,0.232815,0.201953,0.179395,0.493105,0.127016,0.266469,0.240096,0.125594,0.200097,0.152683,0.159806,0.148977,0.099802,0.158911,0.017242,0.240512,0.373664,0.0,0.447089,0.0,0.173678,0.0,0.146909,0.353047,-0.001423,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.290753,0.277549,1.000000,0.090926,0.415104,0.279491,0.399935,0.312458,0.303148,0.237330,0.245375,0.201190,0.142533,0.174239,0.142207,0.289802,0.211008,0.197679,0.275104,0.092660,0.298123,0.248477,0.149460,0.288778,0.210335,0.196814,0.254857,0.094057,0.110663,-0.001079,0.204485,0.281541,0.0,0.214764,0.0,0.286951,0.0,0.097985,0.239915,0.098857,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.032318,0.104439,0.090926,1.000000,0.187172,0.087661,0.274035,0.157461,-0.000962,0.093411,0.152422,-0.000978,0.151466,0.109517,0.156630,0.110912,0.235538,-0.001093,0.072915,-0.000909,0.128145,0.159021,-0.000949,0.130354,0.178020,0.177517,0.128982,0.052620,0.039227,-0.000393,0.280541,0.084029,0.0,0.121925,0.0,0.234066,0.0,-0.000474,0.056167,-0.000350,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.302935,0.283069,0.415104,0.187172,1.000000,0.294428,0.471507,0.282476,0.333483,0.212366,0.268212,0.205894,0.115006,0.253320,0.101963,0.189711,0.272440,0.226073,0.276853,0.049441,0.220088,0.213012,0.161615,0.232784,0.285447,0.215406,0.277869,0.035528,0.139759,-0.001055,0.249277,0.279997,0.0,0.257995,0.0,0.298096,0.0,0.103150,0.241543,-0.000938,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
def suggest_movies(movie_name,user_rating):
    similar_score=its[movie_name]*(user_rating-df5['rating'].mean())
    similar_score=similar_score.sort_values(ascending=False)
    return similar_score[:5]

In [57]:
suggest_movies('movie 0',5)

0       1.498443
3113    0.850011
479     0.833244
779     0.832490
259     0.820176
Name: movie 0, dtype: float64